In [2]:
import numpy as np
import pandas as pd
from helpfiles.temp_save_load import save_files, load_files
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
import helpfiles.ml_pipeline_config as configurations

#Load data
df = pd.read_csv("C:/Users/PC Paul/Downloads/DataEngOwnProject/data/creditcard.csv")#.head(10000) 
while len(df.index) < 1000000:
    df = pd.concat([df,df])
df.reset_index(drop = True, inplace = True)

#scale_data():
rob_scaler = RobustScaler()
df['scaled_amount'] = rob_scaler.fit_transform(df['Amount'].values.reshape(-1,1))
df['scaled_time'] = rob_scaler.fit_transform(df['Time'].values.reshape(-1,1))
df.drop(['Time','Amount'], axis=1, inplace=True)


#split_data_train_val():
split_ratio = configurations.params['test_split_ratio']
# Splitting
X = df.drop('Class', axis=1)
y = df['Class']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)
# Dataframe is concatenated again as the full dataset is needed and split again during cross validation. _val dataframes are needed for the final evaluation. 
df = pd.concat([X_train, y_train],axis = 1)

#undersample_data():
df2 = df.sample(frac=1) # this shuffles the initial df
# Filter fraud cases and concat equal amount of non-fraud-cases
fraud_df = df2.loc[df['Class'] == 1]
non_fraud_df = df2.loc[df['Class'] == 0][:len(fraud_df)]
normal_distributed_df = pd.concat([fraud_df, non_fraud_df])
# Shuffle dataframe rows
new_df = normal_distributed_df.sample(frac=1, random_state=42)

In [30]:
test_observation = new_df.iloc[4:5, :-1].values
test_observation

array([[-2.16992898e+00,  3.63965400e+00, -4.50849779e+00,
         2.73066815e+00, -2.12269287e+00, -2.34101683e+00,
        -4.23525308e+00,  1.70353766e+00, -1.30527913e+00,
        -6.71672002e+00,  6.35361232e+00, -8.60164826e+00,
         4.49930038e-01, -7.50616937e+00, -4.38081782e-01,
        -3.69451600e+00, -6.30475339e+00, -1.26758713e+00,
         3.57987030e-01,  5.00779054e-01,  6.45103276e-01,
        -5.03529449e-01, -5.22821818e-04,  7.16957788e-02,
         9.20074334e-02,  3.08497932e-01,  5.52590915e-01,
         2.98954479e-01, -2.93419030e-01, -8.93773496e-01]])

In [38]:
import joblib
svc.predict(test_observation)


now = datetime.now().strftime('%d-%m-%Y_%H-%M-%S')
filename = 'model_' + '.pkl'
joblib.dump(svc, filename, compress=1)
mj = joblib.load(filename)
mj.predict(test_observation)

array([1], dtype=int64)

In [5]:
#from sklearn.decomposition import PCA, TruncatedSVD

# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
#from sklearn.ensemble import RandomForestClassifier
import collections

# Other Libraries
#from sklearn.model_selection import train_test_split
#from sklearn.pipeline import make_pipeline
#from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
#from imblearn.over_sampling import SMOTE
#from imblearn.under_sampling import NearMiss
#from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime

import helpfiles.ml_pipeline_config as configurations

#experiment():
X = new_df.drop('Class', axis=1)
y = new_df['Class']

#classifiers = {
#"LogisticRegression": LogisticRegression(),
#"Support Vector Classifier": SVC()
#}

#for key, classifier in classifiers.items():
#classifier.fit(X, y)
#training_score = cross_val_score(classifier, X, y, cv=5, scoring = 'f1')
#print("Classifiers: ", classifier.__class__.__name__, "Has a training F1 of", round(training_score.mean(), 2) * 100)


# Logistic Regression 
log_reg_maxiter = configurations.params["logreg_maxiter"]
log_reg_params = {"penalty": ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]} #todo: put this into pipeline config
# Grid search
grid_log_reg = GridSearchCV(LogisticRegression(), log_reg_params, scoring = 'f1')
#grid_log_reg = GridSearchCV(LogisticRegression(max_iter = log_reg_maxiter), log_reg_params) # todo: see whether you can use this line
grid_log_reg.fit(X, y)
# Logistic Regression estimator:
log_reg = grid_log_reg.best_estimator_

# Support Vector Classifier
svc_params = {'C': [0.5, 0.7, 0.9, 1], 'kernel': ['rbf', 'poly', 'sigmoid', 'linear']} #todo: put this into pipeline config
# Grid search
grid_svc = GridSearchCV(SVC(), svc_params, scoring = 'f1')
grid_svc.fit(X, y)
# SVC best estimator
svc = grid_svc.best_estimator_

# Evaluation
# Logistic Regression 
log_reg_score = cross_val_score(log_reg, X, y, cv=5, scoring = 'f1')
print('Logistic Regression Cross Validation F1 score: ', round(log_reg_score.mean() * 100, 2).astype(str) + '%')
print("Best Parameters Logistic Regression: " + str(grid_log_reg.best_params_))
# SVC
svc_score = cross_val_score(svc, X, y, cv=5, scoring = 'f1')
print('Support Vector Classifier Cross Validation F1 score', round(svc_score.mean() * 100, 2).astype(str) + '%')
print("Best Parameters SVM: " + str(grid_svc.best_params_))

#X_val = load_files(['X_val'])[0]
#y_val = load_files(['y_val'])[0]

#Predict Y
y_pred_log_reg = log_reg.predict(X_val)
y_pred_SVM = svc.predict(X_val)
f1_log_reg = f1_score(y_val, y_pred_log_reg)
f1_SVM = f1_score(y_val, y_pred_SVM)

now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

log_reg_results = pd.DataFrame([[
    now,
    'Logistic Regression',
    grid_log_reg.best_estimator_,
    grid_log_reg.best_params_,
    f1_log_reg
    ]],
    columns = [
        'experiment_date',
        'method',
        'best_estimator',
        'best_parameters',
        'f1_score'])

svc_results = pd.DataFrame([[
    now,
    'Support Vector Classifier',
    grid_svc.best_estimator_,
    grid_svc.best_params_,
    f1_SVM
    ]],
    columns = [
        'experiment_date',
        'method',
        'best_estimator',
        'best_parameters',
        'f1_score'])

results = pd.concat([log_reg_results,svc_results])

#Creation of the dataset of evalutation metrics



Logistic Regression Cross Validation F1 score:  94.15%
Best Parameters Logistic Regression: {'C': 10, 'penalty': 'l2'}
Support Vector Classifier Cross Validation F1 score 93.99%
Best Parameters SVM: {'C': 0.5, 'kernel': 'linear'}


In [6]:
results

,experiment_date,method,best_estimator,best_parameters,f1_score
0,2022-03-10_11-22-02,Logistic Regression,LogisticRegression(C=10),"{'C': 10, 'penalty': 'l2'}",0.122163
0,2022-03-10_11-22-02,Support Vector Classifier,"SVC(C=0.5, kernel='linear')","{'C': 0.5, 'kernel': 'linear'}",0.150354


# Second try as a pipeline

In [8]:
# Import necessary packages
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import *

In [9]:
# Initialze the estimators
clf1 = RandomForestClassifier(random_state=42)
clf2 = SVC(probability=True, random_state=42)
clf3 = LogisticRegression(random_state=42)
clf4 = DecisionTreeClassifier(random_state=42)
clf5 = KNeighborsClassifier()
clf6 = MultinomialNB()
clf7 = GradientBoostingClassifier(random_state=42)



# Initiaze the hyperparameters for each dictionary
param1 = {}
param1['classifier__n_estimators'] = [10, 50, 100, 250]
param1['classifier__max_depth'] = [5, 10, 20]
param1['classifier__class_weight'] = [None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}]
param1['classifier'] = [clf1]

param2 = {}
param2['classifier__C'] = [10**-2, 10**-1, 10**0, 10**1, 10**2]
param2['classifier__class_weight'] = [None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}]
param2['classifier'] = [clf2]

param3 = {}
param3['classifier__C'] = [10**-2, 10**-1, 10**0, 10**1, 10**2]
param3['classifier__penalty'] = ['l1', 'l2']
param3['classifier__class_weight'] = [None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}]
param3['classifier'] = [clf3]

param4 = {}
param4['classifier__max_depth'] = [5,10,25,None]
param4['classifier__min_samples_split'] = [2,5,10]
param4['classifier__class_weight'] = [None, {0:1,1:5}, {0:1,1:10}, {0:1,1:25}]
param4['classifier'] = [clf4]

param5 = {}
param5['classifier__n_neighbors'] = [2,5,10,25,50]
param5['classifier'] = [clf5]

param6 = {}
param6['classifier__alpha'] = [10**0, 10**1, 10**2]
param6['classifier'] = [clf6]

param7 = {}
param7['classifier__n_estimators'] = [10, 50, 100, 250]
param7['classifier__max_depth'] = [5, 10, 20]
param7['classifier'] = [clf7]



pipeline = Pipeline([('classifier', clf1)])
params = [param1, param2, param3, param4, param5, param6, param7]



In [10]:


%%time
# Train the grid search model
gs = GridSearchCV(pipeline, params, cv=3, n_jobs=-1, scoring='roc_auc').fit(X,y)



CPU times: total: 5.2 s
Wall time: 1min 17s


In [11]:
gs.best_params_

{'classifier': GradientBoostingClassifier(max_depth=5, random_state=42),
 'classifier__max_depth': 5,
 'classifier__n_estimators': 100}

In [16]:
score_f1 = cross_val_score(gs.best_estimator_, X, y, cv=5, scoring = 'f1')
score_f1.mean()

0.9900020081692865

In [17]:
#Predict Y
y_pred= gs.best_estimator_.predict(X_val)
f1_val = f1_score(y_val, y_pred)
f1_val

0.22436363636363638